# 多输出分类与高级优化

本笔记本介绍了多类分类和多标签分类的区别，以及高级优化算法（如Adam）的实现和应用。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 多类分类 vs 多标签分类

In [ ]:
print("多类分类与多标签分类的区别:")

print("\n多类分类 (Multi-class Classification):")
print("1. 定义: 每个样本属于且仅属于多个类别中的一个")
print("2. 示例: ")
print("   - 图像分类 (猫、狗、鸟)")
print("   - 手写数字识别 (0-9)")
print("   - 情感分析 (积极、中性、消极)")
print("3. 输出层: ")
print("   - 神经元数量 = 类别数量")
print("   - 激活函数: Softmax")
print("   - 输出: 概率分布，和为1")
print("4. 损失函数: 多类交叉熵")

print("\n多标签分类 (Multi-label Classification):")
print("1. 定义: 每个样本可以同时属于多个类别")
print("2. 示例: ")
print("   - 图像标注 (一张图片可以同时包含猫和狗)")
print("   - 文本分类 (一篇文章可以同时属于科技和教育类别)")
print("   - 音乐类型分类 (一首歌可以同时属于摇滚和流行)")
print("3. 输出层: ")
print("   - 神经元数量 = 标签数量")
print("   - 激活函数: Sigmoid (每个标签独立判断)")
print("   - 输出: 每个标签的独立概率")
print("4. 损失函数: 二元交叉熵（每个标签独立计算）")

## 2. 多类分类的实现

In [ ]:
# 多类分类数据生成
def generate_multiclass_data(n_samples=200, n_features=2, n_classes=3):
    """
    生成多类分类数据
    
    参数:
    n_samples: 样本数量
    n_features: 特征维度
    n_classes: 类别数量
    
    返回:
    X: 特征矩阵 (n_samples, n_features)
    y: 标签向量 (n_samples,)
    """
    from sklearn.datasets import make_classification
    
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_features,
        n_redundant=0,
        n_classes=n_classes,
        n_clusters_per_class=1,
        random_state=42
    )
    
    return X, y

# 生成并可视化数据
X, y = generate_multiclass_data(n_samples=200, n_features=2, n_classes=3)

plt.figure(figsize=(8, 6))
for i in range(3):
    plt.scatter(X[y == i, 0], X[y == i, 1], label=f'类别 {i}')
plt.title('多类分类数据集')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.legend()
plt.grid(True)
plt.show()

print(f"数据形状: X={X.shape}, y={y.shape}")
print(f"类别: {np.unique(y)}")

## 3. 多标签分类的实现

In [ ]:
# 多标签分类数据生成
def generate_multilabel_data(n_samples=200, n_features=2, n_labels=3, n_classes_per_sample=2):
    """
    生成多标签分类数据
    
    参数:
    n_samples: 样本数量
    n_features: 特征维度
    n_labels: 标签数量
    n_classes_per_sample: 每个样本平均的标签数量
    
    返回:
    X: 特征矩阵 (n_samples, n_features)
    y: 标签矩阵 (n_samples, n_labels)
    """
    from sklearn.datasets import make_multilabel_classification
    
    X, y = make_multilabel_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_classes=n_labels,
        n_labels=n_classes_per_sample,
        random_state=42
    )
    
    return X, y

# 生成并可视化数据
X_ml, y_ml = generate_multilabel_data(n_samples=200, n_features=2, n_labels=3)

plt.figure(figsize=(12, 4))

# 原始数据分布
plt.subplot(1, 3, 1)
plt.scatter(X_ml[:, 0], X_ml[:, 1])
plt.title('多标签分类数据集')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.grid(True)

# 标签1的分布
plt.subplot(1, 3, 2)
plt.scatter(X_ml[y_ml[:, 0] == 1, 0], X_ml[y_ml[:, 0] == 1, 1], c='r', label='标签 1')
plt.scatter(X_ml[y_ml[:, 0] == 0, 0], X_ml[y_ml[:, 0] == 0, 1], c='gray', alpha=0.3)
plt.title('标签 1 的分布')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.legend()
plt.grid(True)

# 标签2的分布
plt.subplot(1, 3, 3)
plt.scatter(X_ml[y_ml[:, 1] == 1, 0], X_ml[y_ml[:, 1] == 1, 1], c='g', label='标签 2')
plt.scatter(X_ml[y_ml[:, 1] == 0, 0], X_ml[y_ml[:, 1] == 0, 1], c='gray', alpha=0.3)
plt.title('标签 2 的分布')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"数据形状: X={X_ml.shape}, y={y_ml.shape}")
print(f"标签示例:")
print(y_ml[:5])
print(f"每个样本的标签数量:")
print(np.sum(y_ml, axis=1)[:5])

## 4. 高级优化算法：Adam

In [ ]:
print("Adam优化算法:")
print("1. 定义: Adam (Adaptive Moment Estimation) 是一种自适应学习率优化算法")
print("2. 特点: ")
print("   - 结合了AdaGrad和RMSProp的优点")
print("   - 计算每个参数的自适应学习率")
print("   - 利用动量加速收敛")
print("   - 对噪声和稀疏梯度具有鲁棒性")
print("3. 超参数:")
print("   - learning_rate: 学习率 (默认: 0.001)")
print("   - beta1: 一阶动量的指数衰减率 (默认: 0.9)")
print("   - beta2: 二阶动量的指数衰减率 (默认: 0.999)")
print("   - epsilon: 防止除零的小常数 (默认: 1e-7)")
print("4. 优势:")
print("   - 不需要手动调整学习率")
print("   - 收敛速度快")
print("   - 适用于大规模数据和参数")
print("   - 适用于非平稳目标和噪声问题")

## 5. Adam优化算法的实现

In [ ]:
class AdamOptimizer:
    """
    Adam优化器实现
    """
    
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7):
        """
        初始化Adam优化器
        
        参数:
        learning_rate: 学习率
        beta1: 一阶动量的指数衰减率
        beta2: 二阶动量的指数衰减率
        epsilon: 防止除零的小常数
        """
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None  # 一阶动量
        self.v = None  # 二阶动量
        self.t = 0     # 时间步
    
    def update(self, params, gradients):
        """
        使用Adam算法更新参数
        
        参数:
        params: 当前参数
        gradients: 参数的梯度
        
        返回:
        更新后的参数
        """
        # 初始化动量
        if self.m is None:
            self.m = np.zeros_like(params)
            self.v = np.zeros_like(params)
        
        # 时间步增加
        self.t += 1
        
        # 计算一阶动量（带偏置校正）
        self.m = self.beta1 * self.m + (1 - self.beta1) * gradients
        m_hat = self.m / (1 - self.beta1 ** self.t)
        
        # 计算二阶动量（带偏置校正）
        self.v = self.beta2 * self.v + (1 - self.beta2) * (gradients ** 2)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        
        # 更新参数
        params -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
        
        return params

## 6. Adam优化器的测试

In [ ]:
# 测试Adam优化器
def test_function(x):
    """
    测试函数: f(x) = x^2
    最小值在x=0处
    """
    return x ** 2

def test_function_gradient(x):
    """
    测试函数的梯度: f'(x) = 2x
    """
    return 2 * x

# 初始化
x = 5.0  # 初始值
optimizer = AdamOptimizer(learning_rate=0.1)
steps = 20
history = []

# 优化过程
for i in range(steps):
    value = test_function(x)
    history.append(value)
    print(f"Step {i+1}: x = {x:.6f}, f(x) = {value:.6f}")
    
    # 计算梯度
    gradient = test_function_gradient(x)
    
    # 更新参数
    x = optimizer.update(x, gradient)

# 可视化优化过程
plt.figure(figsize=(8, 4))
plt.plot(range(1, steps+1), history)
plt.title('Adam优化器的收敛过程')
plt.xlabel('迭代步数')
plt.ylabel('函数值')
plt.grid(True)
plt.show()

## 7. 额外的层类型

In [ ]:
print("神经网络中常见的层类型:")

print("\n1. 全连接层 (Fully Connected Layer):")
print("   - 每个神经元与前一层的所有神经元相连")
print("   - 参数数量: input_size * output_size + output_size")
print("   - 用途: 特征提取和转换")

print("\n2. 卷积层 (Convolutional Layer):")
print("   - 局部连接，权值共享")
print("   - 适合处理网格数据（图像）")
print("   - 保留空间结构信息")

print("\n3. 池化层 (Pooling Layer):")
print("   - 下采样，减少特征维度")
print("   - 最大池化和平均池化")
print("   - 增加模型的鲁棒性")

print("\n4. 批归一化层 (Batch Normalization Layer):")
print("   - 对每批数据进行归一化")
print("   - 加速训练，减少内部协变量偏移")
print("   - 可以使用更高的学习率")

print("\n5.  dropout层 (Dropout Layer):")
print("   - 随机失活神经元")
print("   - 防止过拟合")
print("   - 模拟集成学习")

print("\n6. 激活层 (Activation Layer):")
print("   - 应用非线性激活函数")
print("   - ReLU、Sigmoid、tanh等")
print("   - 引入非线性能力")

print("\n7. 循环层 (Recurrent Layer):")
print("   - 处理序列数据")
print("   - 包含记忆单元")
print("   - LSTM、GRU等变体")

## 8. 计算图

In [ ]:
print("计算图的基本概念:")
print("1. 定义: 计算图是一种表示数学计算的有向图")
print("2. 组成: ")
print("   - 节点 (Nodes): 表示操作或变量")
print("   - 边 (Edges): 表示数据流向")
print("3. 优势: ")
print("   - 自动微分: 便于梯度计算")
print("   - 并行计算: 识别可并行的操作")
print("   - 内存优化: 高效管理计算资源")
print("   - 代码可读性: 清晰表示计算流程")

print("\n深度学习框架中的计算图:")
print("- TensorFlow: 静态计算图")
print("- PyTorch: 动态计算图")
print("- JAX: 函数式编程风格的计算图")

print("\n计算图的应用:")
print("1. 前向传播: 从输入到输出的计算")
print("2. 反向传播: 从损失到参数的梯度计算")
print("3. 模型优化: 基于梯度的参数更新")
print("4. 模型部署: 导出为可执行格式")

## 9. 更大的神经网络示例

In [ ]:
# 实现一个更大的神经网络，包含批归一化和dropout
class DeepNeuralNetwork:
    """
    深层神经网络实现
    包含批归一化和dropout
    """
    
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.5):
        """
        初始化深层神经网络
        
        参数:
        input_size: 输入特征维度
        hidden_sizes: 隐藏层大小列表
        output_size: 输出维度
        dropout_rate: dropout比率
        """
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        
        # 初始化参数
        self.weights = []
        self.biases = []
        self.batch_norm_params = []
        
        # 输入层到第一个隐藏层
        self.weights.append(np.random.randn(input_size, hidden_sizes[0]) * np.sqrt(2 / input_size))
        self.biases.append(np.zeros(hidden_sizes[0]))
        self.batch_norm_params.append({
            'gamma': np.ones(hidden_sizes[0]),
            'beta': np.zeros(hidden_sizes[0])
        })
        
        # 隐藏层之间
        for i in range(1, len(hidden_sizes)):
            self.weights.append(np.random.randn(hidden_sizes[i-1], hidden_sizes[i]) * np.sqrt(2 / hidden_sizes[i-1]))
            self.biases.append(np.zeros(hidden_sizes[i]))
            self.batch_norm_params.append({
                'gamma': np.ones(hidden_sizes[i]),
                'beta': np.zeros(hidden_sizes[i])
            })
        
        # 最后一个隐藏层到输出层
        self.weights.append(np.random.randn(hidden_sizes[-1], output_size) * np.sqrt(2 / hidden_sizes[-1]))
        self.biases.append(np.zeros(output_size))
    
    def relu(self, x):
        """
        ReLU激活函数
        """
        return np.maximum(0, x)
    
    def softmax(self, x):
        """
        Softmax激活函数
        """
        max_x = np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x - max_x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def batch_norm(self, x, gamma, beta, epsilon=1e-5):
        """
        批归一化
        """
        mean = np.mean(x, axis=0, keepdims=True)
        var = np.var(x, axis=0, keepdims=True)
        x_norm = (x - mean) / np.sqrt(var + epsilon)
        return gamma * x_norm + beta
    
    def dropout(self, x, training=True):
        """
        Dropout层
        """
        if training:
            mask = np.random.binomial(1, 1 - self.dropout_rate, size=x.shape)
            return x * mask / (1 - self.dropout_rate)
        else:
            return x
    
    def forward(self, X, training=True):
        """
        前向传播
        """
        A = X
        
        # 隐藏层
        for i in range(len(self.hidden_sizes)):
            Z = np.dot(A, self.weights[i]) + self.biases[i]
            Z_norm = self.batch_norm(Z, self.batch_norm_params[i]['gamma'], self.batch_norm_params[i]['beta'])
            A = self.relu(Z_norm)
            A = self.dropout(A, training)
        
        # 输出层
        Z = np.dot(A, self.weights[-1]) + self.biases[-1]
        A = self.softmax(Z)
        
        return A
    
    def predict(self, X):
        """
        预测类别
        """
        probabilities = self.forward(X, training=False)
        return np.argmax(probabilities, axis=1)

# 测试深层神经网络
print("测试深层神经网络:")

# 创建示例数据
batch_size = 10
input_size = 10
hidden_sizes = [64, 32, 16]  # 三个隐藏层
output_size = 3  # 3个类别

X = np.random.randn(batch_size, input_size)

# 初始化网络
model = DeepNeuralNetwork(input_size, hidden_sizes, output_size, dropout_rate=0.3)

# 前向传播
probabilities = model.forward(X, training=True)
predictions = model.predict(X)

print(f"输入形状: {X.shape}")
print(f"概率输出形状: {probabilities.shape}")
print(f"预测结果形状: {predictions.shape}")

print("\n概率输出:")
print(probabilities)

print("\n每一行的和:")
print(np.sum(probabilities, axis=1))

print("\n预测结果:")
print(predictions)

## 10. 总结

In [ ]:
print("多输出分类与高级优化总结:")

print("\n1. 多类分类与多标签分类的区别:")
print("   - 多类分类: 每个样本仅属于一个类别，使用Softmax激活")
print("   - 多标签分类: 每个样本可属于多个类别，使用Sigmoid激活")

print("\n2. 高级优化算法:")
print("   - Adam: 结合动量和自适应学习率")
print("   - 优势: 收敛快，不需要手动调整学习率")
print("   - 超参数: 学习率、beta1、beta2、epsilon")

print("\n3. 额外的层类型:")
print("   - 全连接层: 基本的特征转换")
print("   - 卷积层: 处理网格数据")
print("   - 池化层: 下采样")
print("   - 批归一化层: 加速训练")
print("   - Dropout层: 防止过拟合")

print("\n4. 计算图:")
print("   - 表示计算流程的有向图")
print("   - 便于自动微分和并行计算")
print("   - 深度学习框架的核心概念")

print("\n5. 更大的神经网络:")
print("   - 多层结构: 捕获更复杂的特征")
print("   - 批归一化: 加速深层网络训练")
print("   - Dropout: 提高模型泛化能力")
print("   - 适当的初始化: 避免梯度消失/爆炸")